In [95]:
!pip install PyGithub

# Package Imports
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from urllib.request import urlopen # The default requests package
import requests # For making GitHub requests
from pprint import pprint # For pretty printing
# from getpass import getpass # For keeping password typing a secret

# For the more advanced requests
import base64
import os
import sys
sys.path.append("./PyGithub");
from github import Github
from getpass import getpass

In [188]:
# Recursively Import the Data (AUTOMATIC)

g = Github(getpass("Enter your PAT key")) # Enter your PAT Key. A system which is so stupidly unsafe it's mindboggling

user = g.get_user(username)
for repo in g.get_user().get_repos():
    if repo.name == "ai-group-project-Team-JMJM":
      r_proj_clone = repo
      break
    # To see all the available attributes and methods
    print(dir(repo))

contents = repo.get_contents("RapLyrics/UNCLEAN", ref='PROTOTYPE')

RAP_DATA = []
rap_lyric_names = []

for file_ in contents:
  path = file_.path
  path = str(path) 
  # Only choose the .txt files
  if path[-4:] == '.txt':
    # Append the name
    title_start = path_.find('UNCLEAN')
    title_len = path[title_start:].find('.')
    name = path[title_start + 8:title_start + title_len]
    if name[-2:] == 'UC':
      name = name[:-2]
    rap_lyric_names.append(name) 

    # Append the Lyrics
    RAP_DATA.append(file_.decoded_content.decode("utf-8")) 
  elif path == 'RapLyrics/UNCLEAN/censors.csv':
    censors = file_.decoded_content

    # Split the long string into a list of lines, then split by words, then put into a csv, then to numpy arr
    data = censors.decode('utf-8')[1:].splitlines()
    data_rows = []
    for count, word in enumerate(data):
      if count>0:
        data_rows.append(word.split(','))
    censors = pd.DataFrame(data_rows)
    censors = censors.to_numpy()
    
# Remove the \ufeff at the beginning O(n)
for count, lyric in enumerate(RAP_DATA): 
  RAP_DATA[count] = lyric[1:]

# Censor the profanities O(n*m)
for count in range(len(RAP_DATA)): 
  for i in range(len(censors[0:])):
    RAP_DATA[count] = RAP_DATA[count].replace(str(censors[i, 0]), str(censors[i, 1]))

contents = repo.get_contents("RapLyrics/CLEAN", ref='PROTOTYPE')
cleaned_names = []
for file_ in contents:
  path = file_.path
  path = str(path) 
  print(path)
  # Only choose the .txt files
  if path[-4:] == '.txt':
    # Append the name
    title_start = path_.find('CLEAN')
    title_len = path[title_start:].find('.')
    name = path[title_start + 4:title_start + title_len]
    if name[-2:] == 'CL':
      name = name[:-2]
    cleaned_names.append(name) 

# ALL OF THE EDITING IS DONE IN THE 'PROTOTYPE BRANCH'
# If the (now cleaned) rap_lyrics name is new (not in cleaned_names), then we want to create that as a new file 
# If the (now cleaned) rap_lyrics name is NOT new (not in cleaned_names), then we want to update the file
print(rap_lyric_names)
print(cleaned_names)

for counter, new_name in enumerate(rap_lyric_names): 
  if new_name in cleaned_names: 
    duplicate = repo.get_contents("RapLyrics/CLEAN/{}CL.txt".format(new_name), ref='PROTOTYPE')
    repo.update_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], duplicate.sha, branch='PROTOTYPE')
  else:
    repo.create_file("RapLyrics/CLEAN/{}CL.txt".format(new_name), "This was uploaded automatically via pipeline", RAP_DATA[counter], branch="PROTOTYPE")

RapLyrics/CLEAN/Kanye_WestCL.txt
RapLyrics/CLEAN/Lil_WayneCL.txt
['Kanye_West', 'Lil_Wayne', 'eminem', 'ludacris', 'nicki-minaj', 'notorious-big']
['Kanye_West', 'Lil_Wayne']


In [ ]:
# Data Imports (MANUAL and requires publicising the project)
URLS = ['https://raw.githubusercontent.com/EMAT31530/ai-group-project-Team-JMJM/master/RapLyrics/UNCLEAN/Kanye_WestUC.txt']
censors = pd.read_csv("https://raw.githubusercontent.com/EMAT31530/ai-group-project-Team-JMJM/master/RapLyrics/UNCLEAN/censors.csv").to_numpy()
RAP_DATA = []
rap_lyric_names = []
for URL in URLS: 
  title_start = URL.find('UNCLEAN')
  title_len = URL[title_start:].find('.')
  rap_lyric_names.append(URL[title_start + 8:title_start + title_len])

# Import all of the data through URLS O(n)
for URL in URLS: 
  RAP_DATA.append(urlopen(URL).read().decode("utf-8"))

# Remove the \ufeff at the beginning O(n)
for count, lyric in enumerate(RAP_DATA): 
  RAP_DATA[count] = lyric[1:]

# Censor the profanities O(n*m)
for count, lyric in enumerate(RAP_DATA): 
  for i in range(len(censors[0:])):
    RAP_DATA[count] = lyric.replace(censors[i, 0], censors[i, 1])

print(RAP_DATA[0])